# Building a Model Experimentation Lineage with Pre-Built Code

## Learning objectives

1. Create a Cloud Storage bucket.
2. Model experimentation with pre-built data pre-processing code.
3. Visualize Experiment Lineage.


## Introduction 

In this notebook, you learn how to integrate preprocessing code in a Vertex AI experiments. Also you will build the experiment lineage lets you record, analyze, debug, and audit metadata and artifacts produced along your ML journey.

### Dataset

This dataset is the UCI News Aggregator Data Set which contains 422,937 news collected between March 10th, 2014 and August 10th, 2014. Below are example records from the dataset:


|ID |TITLE                                                                |URL                                                                                                                        |PUBLISHER        |CATEGORY|STORY                        |HOSTNAME           |TIMESTAMP    |
|---|---------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-----------------|--------|-----------------------------|-------------------|-------------|
|1  |Fed official says weak data caused by weather, should not slow taper |http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss|Los Angeles Times|b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.latimes.com    |1394470370698|
|2  |Fed's Charles Plosser sees high bar for change in pace of tapering   |http://www.livemint.com/Politics/H2EvwJSK2VE6OF7iK1g3PP/Feds-Charles-Plosser-sees-high-bar-for-change-in-pace-of-ta.html   |Livemint         |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.livemint.com   |1394470371207|
|3  |US open: Stocks fall after Fed official hints at accelerated tapering|http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436                |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371550|
|4  |Fed risks falling 'behind the curve', Charles Plosser says           |http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-plosser-says-294430                             |IFA Magazine     |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371793|
|5  |Fed's Plosser: Nasty Weather Has Curbed Job Growth                   |http://www.moneynews.com/Economy/federal-reserve-charles-plosser-weather-job-growth/2014/03/10/id/557011                   |Moneynews        |b       |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.moneynews.com  |1394470372027|





Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first before reviewing this [solution notebook](../solutions/build_model_experimentation_lineage_with_prebuild_code.ipynb).



### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as TensorFlow or Vertex AI SDK. Use the latest major GA version of each package.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade joblib fsspec gcsfs scikit-learn -q
! pip3 install {USER_FLAG} --force-reinstall 'google-cloud-aiplatform>=1.15' -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.18.0 requires tensorflow-io-gcs-filesystem==0.18.0, which is not installed.
tfx-bsl 1.2.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.13.0 which is incompatible.
tfx-bsl 1.2.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.15.0 which is incompatible.
tfx-bsl 1.2.0 requires google-cloud-bigquery<2.21,>=1.28.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
tfx-bsl 1.2.0 requires pyarrow<3,>=1, 

**Note**: Please ignore the incompatibility warnings and errors.

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = "qwiklabs-gcp-04-c11d0df894dc"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-04-c11d0df894dc


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "qwiklabs-gcp-04-c11d0df894dc"  # @param {type:"string"}

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [6]:
# import necessary libraries

import os
import sys

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [7]:
BUCKET_NAME = "qwiklabs-gcp-04-c11d0df894dc"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [8]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://qwiklabs-gcp-04-c11d0df894dc/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [10]:
! gsutil ls -al $BUCKET_URI

### Set project folder

In [11]:
DATA_PATH = "data"

In [12]:
!mkdir -m 777 -p {DATA_PATH}

### Get the data

In [13]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"

In [14]:
!wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
!mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
!unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
!mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip

--2022-08-23 10:36:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘data/NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  35.0MB/s    in 0.8s    

2022-08-23 10:36:33 (35.0 MB/s) - ‘data/NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  data/NewsAggregatorDataset.zip
  inflating: data/temp/2pageSessions.csv  
   creating: data/temp/__MACOSX/
  inflating: data/temp/__MACOSX/._2pageSessions.csv  
  inflating: data/temp/newsCorpora.csv  
  inflating: data/temp/__MACOSX/._newsCorpora.csv  
  inflating: data/temp/readme.txt    
  inflating: data/temp/__MACOSX/._readme.txt  


In [15]:
!gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

Copying file://data/raw/newsCorpora.csv [Content-Type=text/csv]...
Copying file://data/raw/2pageSessions.csv [Content-Type=text/csv]...            
- [2/2 files][100.5 MiB/100.5 MiB] 100% Done                                    
Operation completed over 2 objects/100.5 MiB.                                    


### Import libraries

In [16]:
# General
import logging

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import collections
import tempfile
import time
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Define constants

In [17]:
# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{TIMESTAMP}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{TIMESTAMP}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [18]:
# Initialize the vertex AI
# TODO: Your code goes here (
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

INFO:root:MetadataStore default not found.


### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [19]:
SERVE_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn", framework_version="1.0", accelerator="cpu"
)

### Initialize Experiment run

In [20]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

Associating projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1 to Experiment: classification-naivebayes-20220823103532


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1 to Experiment: classification-naivebayes-20220823103532


## Model experimentation with pre-built data pre-processing code

### Data preprocessing

In this step, you simulate some preprocessing on raw data to create the training dataset.

Indeed it would happen that you have some data preprocessing that others within your team may have written. For that reason, you need a way to integrate the preprocessing code in your experiment runs to standardize and reuse all the complex data wrangling that you are dealing with. 

Using Vertex AI Experiments, you can track that code as part of the run execution by adding one `with` statement in your experiment context. 

#### Create a Dataset Metadata Artifact

First you create the Dataset artifact to track the dataset resource in the Vertex AI ML Metadata and create the experiment lineage.

In [21]:
# Use the vertex AI artifact create function
raw_dataset_artifact = # TODO: Your code goew here (
    schema_title="system.Dataset", display_name=DATASET_NAME, uri=DATASET_URI
)

#### Create a preprocess module

Next, you build a simple preprocessing module to convert the text case and remove the punctuation.

In [22]:
"""
Preprocess module
"""

import string

import pandas as pd


def preprocess(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """
    Preprocess text
    Args:
        df: The DataFrame to preprocesss
        text_col: The text column name
    Returns:
        preprocessed_df: The datafrane with text in lowercase and without punctuation
    """
    preprocessed_df = df.copy()
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(lambda x: x.lower())
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(
        lambda x: x.translate(str.maketrans("", "", string.punctuation))
    )
    return preprocessed_df

#### Add the `preprocessing` Execution

You add the preprocessing step to track its execution in the lineage associated to Vertex AI Experiment. 

In [23]:
# Use the vertex AI start execution function
with # TODO: Your code goes here (
    schema_title="system.ContainerExecution", display_name=PREPROCESS_EXECUTION_NAME
) as exc:
    logging.info(f"Start {PREPROCESS_EXECUTION_NAME} execution.")
    exc.assign_input_artifacts([raw_dataset_artifact])

    # Log preprocessing params --------------------------------------------------
    logging.info("Log preprocessing params.")
    vertex_ai.log_params(
        {
            "delimiter": DELIMITER,
            "features": dumps(COLUMN_NAMES),
            "index_col": INDEX_COL,
        }
    )

    # Preprocessing ------------------------------------------------------------
    logging.info("Preprocessing.")
    raw_df = pd.read_csv(
        raw_dataset_artifact.uri,
        delimiter=DELIMITER,
        names=COLUMN_NAMES,
        index_col=INDEX_COL,
    )
    preprocessed_df = preprocess(raw_df, "title")
    preprocessed_df.to_csv(PREPROCESSED_DATASET_URI, sep=",")

    # Log preprocessing metrics and store dataset artifact ---------------------
    logging.info(f"Log preprocessing metrics and {PREPROCESSED_DATASET_NAME} dataset.")
    vertex_ai.log_metrics(
        {
            "n_records": preprocessed_df.shape[0],
            "n_columns": preprocessed_df.shape[1],
        },
    )

    preprocessed_dataset_metadata = vertex_ai.Artifact.create(
        schema_title="system.Dataset",
        display_name=PREPROCESSED_DATASET_NAME,
        uri=PREPROCESSED_DATASET_URI,
    )
    exc.assign_output_artifacts([preprocessed_dataset_metadata])

INFO:root:Start preprocess execution.
INFO:root:Log preprocessing params.
INFO:root:Preprocessing.
INFO:root:Log preprocessing metrics and preprocessed_news_corpora dataset.


### Model Training

In this step, you train a multinomial Naive Bayes pipeline.

#### Create model training module

Below the training module.

In [24]:
"""
Train module
"""

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


def get_training_split(
    x: pd.DataFrame, y: pd.Series, test_size: float, random_state: int
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    """
    Splits data into training and testing sets
    Args:
        x: The data to be split
        y: The labels to be split
        test_size: The proportion of the data to be reserved for testing
        random_state: The seed used by the random number generator
    Returns:
        x_train: The training data
        x_test: The testing data
        y_train: The training labels
        y_test: The testing labels
    """

    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    return x_train, x_val, y_train, y_val


def get_pipeline():
    """
    Get the model
    Args:
        None
    Returns:
        model: The model
    """
    model = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", MultinomialNB()),
        ]
    )
    return model


def train_pipeline(model: Pipeline, X_train: pd.Series, y_train: pd.Series) -> Pipeline:
    """
    Train the model
    Args:
        model: The model to train
        X_train: The training data
        y_train: The training labels
    Returns:
        model: The trained model
    """
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: Pipeline, X_test: pd.Series, y_test: pd.Series) -> float:
    """
    Evaluate the model
    Args:
        model: The model to evaluate
        X_test: The testing data
        y_test: The testing labels
    Returns:
        score: The accuracy of the model
    """
    # Evaluate model
    y_pred = model.predict(X_test)

    # Store evaluation metrics
    # Store evaluation metrics
    metrics = {
        "accuracy": round(accuracy_score(y_test, y_pred), 5),
        "precision": round(precision_score(y_test, y_pred, average="weighted"), 5),
        "recall": round(recall_score(y_test, y_pred, average="weighted"), 5),
    }
    return metrics


def save_model(model: Pipeline, save_path: str) -> int:
    try:
        with tempfile.NamedTemporaryFile() as tmp:
            joblib.dump(trained_pipeline, filename=tmp.name)
            ! gsutil cp {tmp.name} {save_path}/model.joblib
    except RuntimeError as error:
        print(error)
    return 1

#### Add the `train` Execution

Next, you add the training task to the experiment execution to update the experiment lineage.

In [25]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=TRAIN_EXECUTION_NAME
) as exc:

    exc.assign_input_artifacts([preprocessed_dataset_metadata])

    # Get training and testing data
    logging.info("Get training and testing data.")
    x_train, x_val, y_train, y_val = get_training_split(
        preprocessed_df[FEATURES],
        preprocessed_df[TARGET],
        test_size=TEST_SIZE,
        random_state=SEED,
    )
    # Get model pipeline
    logging.info("Get model pipeline.")
    pipeline = get_pipeline()

    # Log training param -------------------------------------------------------

    # Log data parameters
    logging.info("Log data parameters.")
    vertex_ai.log_params(
        {
            "target": TARGET,
            "features": FEATURES,
            "test_size": TEST_SIZE,
            "random_state": SEED,
        }
    )

    # Log pipeline parameters
    logging.info("Log pipeline parameters.")
    vertex_ai.log_params(
        {
            "pipeline_steps": dumps(
                {step[0]: str(step[1].__class__.__name__) for step in pipeline.steps}
            )
        }
    )

    # Training -----------------------------------------------------------------

    # Train model pipeline
    logging.info("Train model pipeline.")
    train_start = time.time()
    trained_pipeline = train_pipeline(pipeline, x_train, y_train)
    train_end = time.time()

    # Evaluate model
    logging.info("Evaluate model.")
    model_metrics = evaluate_model(trained_pipeline, x_val, y_val)

    # Log training metrics and store model artifact ----------------------------

    # Log training metrics
    logging.info("Log training metrics.")
    vertex_ai.log_metrics(model_metrics)

    # Generate first ten predictions
    logging.info("Generate prediction sample.")
    prediction_sample = trained_pipeline.predict(x_val)[:10]
    print("prediction sample:", prediction_sample)

    # Upload Model on Vertex AI
    logging.info("Upload Model on Vertex AI.")
    loaded = save_model(trained_pipeline, TRAINED_MODEL_URI)
    if loaded:
        model = vertex_ai.Model.upload(
            serving_container_image_uri=SERVE_IMAGE,
            artifact_uri=TRAINED_MODEL_URI,
            display_name=MODEL_NAME,
        )

    exc.assign_output_artifacts([model])

INFO:root:Get training and testing data.
INFO:root:Get model pipeline.
INFO:root:Log data parameters.
INFO:root:Log pipeline parameters.
INFO:root:Train model pipeline.
INFO:root:Evaluate model.
INFO:root:Log training metrics.
INFO:root:Generate prediction sample.
INFO:root:Upload Model on Vertex AI.


prediction sample: ['m' 'e' 'e' 't' 'e' 'b' 'b' 'e' 'm' 't']
Copying file:///tmp/tmps2wwgyl6 [Content-Type=application/octet-stream]...
/ [1 files][  6.3 MiB/  6.3 MiB]                                                
Operation completed over 1 objects/6.3 MiB.                                      
Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/135696800439/locations/us-central1/models/3065944193588264960/operations/2276996565744746496


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/135696800439/locations/us-central1/models/3065944193588264960/operations/2276996565744746496


...

INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/135696800439/locations/us-central1/models/3065944193588264960@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/135696800439/locations/us-central1/models/3065944193588264960@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/135696800439/locations/us-central1/models/3065944193588264960@1')


In [26]:
run.end_run()

### Visualize Experiment Lineage

Below you will get the link to Vertex AI Metadata UI in the console that will show the experiment lineage.

In [27]:
print("Open the following link:", exc.get_output_artifacts()[0].lineage_console_uri)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/f629492a-3095-4a9f-9600-ab74a38150c3?project=qwiklabs-gcp-04-c11d0df894dc


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [28]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

Deleting Context : projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1


INFO:google.cloud.aiplatform.base:Deleting Context : projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1


Delete Context  backing LRO: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1/operations/7339042546909184000


INFO:google.cloud.aiplatform.base:Delete Context  backing LRO: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1/operations/7339042546909184000


Context deleted. . Resource name: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532-run-1


...

INFO:google.cloud.aiplatform.base:Deleting Context : projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532


Delete Context  backing LRO: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532/operations/1552854015803392


INFO:google.cloud.aiplatform.base:Delete Context  backing LRO: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532/operations/1552854015803392


Context deleted. . Resource name: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532


INFO:google.cloud.aiplatform.base:Context deleted. . Resource name: projects/135696800439/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-20220823103532


In [29]:
# Delete model
model_list = vertex_ai.Model.list(filter=f'display_name="{MODEL_NAME}"')
for model in model_list:
    model.delete()

# Delete dataset
for dataset_name in [DATASET_NAME, PREPROCESSED_DATASET_NAME]:
    dataset_list = vertex_ai.TabularDataset.list(
        filter=f'display_name="{dataset_name}"'
    )
    for dataset in dataset_list:
        dataset.delete()

Deleting Model : projects/135696800439/locations/us-central1/models/3065944193588264960


INFO:google.cloud.aiplatform.base:Deleting Model : projects/135696800439/locations/us-central1/models/3065944193588264960


Delete Model  backing LRO: projects/135696800439/locations/us-central1/operations/6514883815100383232


INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/135696800439/locations/us-central1/operations/6514883815100383232


Model deleted. . Resource name: projects/135696800439/locations/us-central1/models/3065944193588264960


INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/135696800439/locations/us-central1/models/3065944193588264960


In [30]:
# Delete Cloud Storage objects that were created
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

Removing gs://qwiklabs-gcp-04-c11d0df894dc/data/preprocess/preprocessed_news_corpora.csv#1661251081564100...
Removing gs://qwiklabs-gcp-04-c11d0df894dc/data/raw/2pageSessions.csv#1661251002815678...
Removing gs://qwiklabs-gcp-04-c11d0df894dc/data/raw/newsCorpora.csv#1661251003714955...
Removing gs://qwiklabs-gcp-04-c11d0df894dc/deliverables/20220823103532/model.joblib#1661251212765094...
/ [4/4 objects] 100% Done                                                       
Operation completed over 4 objects.                                              
Removing gs://qwiklabs-gcp-04-c11d0df894dc/...


In [31]:
!rm -Rf {DATA_PATH}